In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import folium
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.style.use('seaborn-v0_8-whitegrid')  # 使用seaborn样式

# 创建中文字体对象
font_path = 'C:/Windows/Fonts/simhei.ttf'
if os.path.exists(font_path):
    chinese_font = fm.FontProperties(fname=font_path)
    print("使用SimHei字体文件路径")
else:
    # 回退到rcParams方法
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    chinese_font = None
    print("使用rcParams设置SimHei字体")

# 创建输出目录
output_dir = 'dengue_risk_analysis_results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 读取处理过的数据
print("=== 开始加载数据 ===")
df = pd.read_csv('../pre_data/processed_dengue_data.csv')

# 确保日期列为datetime类型
df['Date'] = pd.to_datetime(df['Date'])

print(f"原始数据形状: {df.shape}")
print(f"数据时间范围: {df['Date'].min()} 至 {df['Date'].max()}")
print("数据加载完成！\n")

使用SimHei字体文件路径
=== 开始加载数据 ===
原始数据形状: (1020, 8)
数据时间范围: 2016-01-01 00:00:00 至 2020-12-01 00:00:00
数据加载完成！



In [43]:
# 1. 创建区域风险评分系统
print("1. 创建区域风险评分系统")

# 1.1 计算各地区总体统计数据
print("  计算地区总体统计数据...")
region_stats = df.groupby('Region').agg({
    'Dengue_Cases': ['sum', 'mean', 'std', 'max'],
    'Dengue_Deaths': ['sum', 'mean']
})

# 重命名列，使其更容易访问
region_stats.columns = [f"{col[0]}_{col[1]}" for col in region_stats.columns]
region_stats = region_stats.reset_index()

# 1.2 计算病死率
print("  计算病死率...")
region_stats['Fatality_Rate'] = (region_stats['Dengue_Deaths_sum'] / region_stats['Dengue_Cases_sum']) * 100

# 1.3 识别季节性高发期
print("  识别季节性高发期...")
# 计算每个地区每个月的平均病例数
monthly_region_cases = df.groupby(['Region', 'MonthNum'])['Dengue_Cases'].mean().reset_index()

# 找出每个地区的高发月份（平均病例数最高的3个月）
high_season_months = monthly_region_cases.sort_values(['Region', 'Dengue_Cases'], ascending=[True, False])
high_season_months = high_season_months.groupby('Region').head(3)

# 计算高发季节的平均病例数
high_season_avg = high_season_months.groupby('Region')['Dengue_Cases'].mean().reset_index()
high_season_avg.columns = ['Region', 'High_Season_Avg_Cases']

# 合并到地区统计数据
region_stats = pd.merge(region_stats, high_season_avg, on='Region')

# 1.4 计算爆发频率
print("  计算爆发频率...")
# 使用95%分位数定义爆发
percentile_95 = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.quantile(0.95))
df['Outbreak'] = (df['Dengue_Cases'] > percentile_95).astype(int)

# 计算每个地区的爆发频率
outbreak_freq = df.groupby('Region')['Outbreak'].mean().reset_index()
outbreak_freq.columns = ['Region', 'Outbreak_Frequency']

# 合并到地区统计数据
region_stats = pd.merge(region_stats, outbreak_freq, on='Region')

# 1.5 归一化指标
print("  归一化风险指标...")
# 选择用于风险评分的指标
risk_factors = [
    'Dengue_Cases_mean',  # 平均病例数
    'Dengue_Cases_max',   # 最大病例数
    'Dengue_Cases_std',   # 病例数标准差（波动性）
    'Fatality_Rate',      # 病死率
    'High_Season_Avg_Cases',  # 高发季平均病例数
    'Outbreak_Frequency'  # 爆发频率
]

# 创建一个临时DataFrame用于标准化
risk_df = region_stats[['Region'] + risk_factors].copy()

# 使用Min-Max缩放将所有指标归一化到0-1范围
scaler = MinMaxScaler()
risk_df[risk_factors] = scaler.fit_transform(risk_df[risk_factors])

# 1.6 计算综合风险评分
print("  计算综合风险评分...")
# 定义各项指标的权重
weights = {
    'Dengue_Cases_mean': 0.2,       # 平均病例数 - 20%
    'Dengue_Cases_max': 0.15,        # 最大病例数 - 15%
    'Dengue_Cases_std': 0.1,        # 波动性 - 10%
    'Fatality_Rate': 0.25,          # 病死率 - 25%
    'High_Season_Avg_Cases': 0.15,  # 高发季严重度 - 15%
    'Outbreak_Frequency': 0.15      # 爆发频率 - 15%
}

# 计算加权风险评分
risk_df['Risk_Score'] = sum(risk_df[factor] * weight for factor, weight in weights.items())

# 将风险评分归一化到0-100
risk_df['Risk_Score'] = risk_df['Risk_Score'] * 100

# 根据风险评分分类
risk_categories = [
    (0, 20, '极低风险'),
    (20, 40, '低风险'),
    (40, 60, '中等风险'),
    (60, 80, '高风险'),
    (80, 100, '极高风险')
]

def get_risk_category(score):
    for low, high, category in risk_categories:
        if low <= score < high:
            return category
    return '极高风险'  # 如果分数是100

risk_df['Risk_Category'] = risk_df['Risk_Score'].apply(get_risk_category)

# 将风险评分结果合并回区域统计数据
region_stats = pd.merge(region_stats, 
                        risk_df[['Region', 'Risk_Score', 'Risk_Category']], 
                        on='Region')

# 按风险评分排序
region_stats = region_stats.sort_values('Risk_Score', ascending=False)

print("  风险评分完成. 前5个高风险地区:")
print(region_stats[['Region', 'Risk_Score', 'Risk_Category']].head(5))

# 1.7 保存风险评分结果
region_stats.to_csv(f'{output_dir}/region_risk_scores.csv', index=False)
print(f"  风险评分结果已保存至: {output_dir}/region_risk_scores.csv")

# 1.8 可视化风险评分分布
print("  可视化风险评分分布...")
plt.figure(figsize=(12, 8))

# 绘制风险评分条形图
bars = plt.barh(region_stats['Region'], region_stats['Risk_Score'], 
                color=[plt.cm.RdYlGn_r(score/100) for score in region_stats['Risk_Score']])

# 添加风险类别标签
for i, (region, score, category) in enumerate(zip(region_stats['Region'], 
                                                 region_stats['Risk_Score'], 
                                                 region_stats['Risk_Category'])):
    plt.text(score + 1, i, f"{score:.1f} ({category})", va='center', fontproperties=chinese_font)

if chinese_font is not None:
    plt.title('菲律宾各地区登革热风险评分', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('风险评分 (0-100)', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('地区', fontproperties=chinese_font, fontsize=12)
else:
    plt.title('菲律宾各地区登革热风险评分', fontsize=16, fontproperties=chinese_font)
    plt.xlabel('风险评分 (0-100)', fontsize=12, fontproperties=chinese_font)
    plt.ylabel('地区', fontsize=12, fontproperties=chinese_font)

plt.xlim(0, 110)  # 保留一些空间给标签
plt.grid(True, axis='x')
plt.tight_layout()
plt.savefig(f'{output_dir}/region_risk_scores.png', dpi=300)
print(f"  保存图表：{output_dir}/region_risk_scores.png")
plt.close()



1. 创建区域风险评分系统
  计算地区总体统计数据...
  计算病死率...
  识别季节性高发期...
  计算爆发频率...
  归一化风险指标...
  计算综合风险评分...
  风险评分完成. 前5个高风险地区:
         Region  Risk_Score Risk_Category
6   Region IV-A   60.760408           高风险
10    Region VI   56.264143          中等风险
2           NCR   54.434713          中等风险
15   Region XII   38.887689           低风险
5    Region III   38.634201           低风险
  风险评分结果已保存至: dengue_risk_analysis_results/region_risk_scores.csv
  可视化风险评分分布...
  保存图表：dengue_risk_analysis_results/region_risk_scores.png


In [44]:
# 2. 实现K-means聚类分析
print("\n2. 实现K-means聚类分析")

# 2.1 准备聚类特征
print("  准备聚类特征...")
# 选择用于聚类的特征
cluster_features = [
    'Dengue_Cases_mean',  # 平均病例数
    'Dengue_Cases_std',   # 病例数标准差（波动性）
    'Dengue_Deaths_mean', # 平均死亡数
    'Fatality_Rate',      # 病死率
    'High_Season_Avg_Cases',  # 高发季平均病例数
    'Outbreak_Frequency'  # 爆发频率
]

# 提取特征矩阵并标准化
X = region_stats[cluster_features].values
X_std = StandardScaler().fit_transform(X)

# 2.2 找到最佳聚类数
print("  确定最佳聚类数...")
# 使用肘部法则和轮廓分数
inertias = []
silhouette_scores = []
k_range = range(2, 8)  # 尝试2到7个聚类

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_std)
    inertias.append(kmeans.inertia_)
    
    # 只有当聚类数大于1时才计算轮廓分数
    if k > 1:
        labels = kmeans.labels_
        silhouette_scores.append(silhouette_score(X_std, labels))

# 可视化肘部法则
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(k_range, inertias, 'bo-')
plt.xlabel('聚类数 (k)', fontproperties=chinese_font)
plt.ylabel('惯性 (总平方距离)', fontproperties=chinese_font)
plt.title('肘部法则', fontproperties=chinese_font)
plt.grid(True)

# 可视化轮廓分数
plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_scores, 'ro-')
plt.xlabel('聚类数 (k)', fontproperties=chinese_font)
plt.ylabel('轮廓分数', fontproperties=chinese_font)
plt.title('不同聚类数的轮廓分数比较', fontproperties=chinese_font)
plt.grid(True)

plt.tight_layout()
plt.savefig(f'{output_dir}/optimal_k_selection.png', dpi=300)
print(f"  保存图表：{output_dir}/optimal_k_selection.png")
plt.close()

# 根据可视化结果选择最佳聚类数
# 这里假设最佳聚类数为4，实际应根据图表结果确定
best_k = 4
print(f"  选择的最佳聚类数: {best_k}")

# 2.3 执行最终K-means聚类
print("  执行最终K-means聚类...")
final_kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
region_stats['Cluster'] = final_kmeans.fit_predict(X_std)

# 2.4 保存聚类结果
region_stats.to_csv(f'{output_dir}/region_clusters.csv', index=False)
print(f"  聚类结果已保存至: {output_dir}/region_clusters.csv")


2. 实现K-means聚类分析
  准备聚类特征...
  确定最佳聚类数...
  保存图表：dengue_risk_analysis_results/optimal_k_selection.png
  选择的最佳聚类数: 4
  执行最终K-means聚类...
  聚类结果已保存至: dengue_risk_analysis_results/region_clusters.csv


In [45]:
# 3. 分析不同聚类的特点
print("\n3. 分析不同聚类的特点")

# 3.1 计算每个聚类的平均特征值
print("  计算聚类特征...")
cluster_means = region_stats.groupby('Cluster')[cluster_features + ['Risk_Score']].mean()
print("  聚类平均特征值:")
print(cluster_means)

# 3.2 可视化聚类特征
print("  可视化聚类特征...")
# 雷达图展示每个聚类的特征
def radar_plot(cluster_data, features, cluster_names, colors, output_filename):
    """创建聚类特征的雷达图"""
    # 确保只使用指定的特征
    cluster_data_selected = cluster_data[features].copy()
    
    # 准备角度
    angles = np.linspace(0, 2*np.pi, len(features), endpoint=False).tolist()
    angles += angles[:1]  # 闭合雷达图
    
    # 准备特征名称
    feature_names = [f.replace('_', ' ').title() for f in features]
    
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    for i, cluster in enumerate(cluster_data_selected.index):
        values = cluster_data_selected.loc[cluster].values.tolist()
        values += values[:1]  # 闭合雷达图
        
        # 确保angles和values长度相同（调试用）
        assert len(angles) == len(values), f"角度长度({len(angles)})与值长度({len(values)})不匹配"
        
        ax.plot(angles, values, 'o-', linewidth=2, label=cluster_names[i], color=colors[i])
        ax.fill(angles, values, alpha=0.1, color=colors[i])
    
    # 设置雷达图属性
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_thetagrids(np.degrees(angles[:-1]), feature_names)
    
    if chinese_font is not None:
        plt.title('聚类特征雷达图', fontproperties=chinese_font, fontsize=16)
        plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1), prop=chinese_font)
    else:
        plt.title('聚类特征雷达图', fontsize=16)
        plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    
    plt.tight_layout()
    plt.savefig(output_filename, dpi=300)
    plt.close()

# 归一化聚类特征用于雷达图 - 修正了这里只使用cluster_features
radar_features = cluster_means[cluster_features].copy()  # 只使用指定的特征
radar_features = (radar_features - radar_features.min()) / (radar_features.max() - radar_features.min())

# 为每个聚类定义一个名称和颜色
cluster_names = [f'聚类 {i}' for i in range(best_k)]
cluster_colors = plt.cm.tab10(np.linspace(0, 1, best_k))

# 创建雷达图
radar_plot(radar_features, cluster_features, cluster_names, cluster_colors, 
           f'{output_dir}/cluster_radar_chart.png')
print(f"  保存图表：{output_dir}/cluster_radar_chart.png")

# 3.3 可视化聚类分布
print("  可视化聚类分布...")
# 使用PCA将特征降至2维以便可视化
pca = PCA(n_components=2)
region_pca = pca.fit_transform(X_std)

plt.figure(figsize=(12, 10))
scatter = plt.scatter(region_pca[:, 0], region_pca[:, 1], 
                     c=region_stats['Cluster'], cmap='tab10', 
                     s=100, alpha=0.8)

# 添加地区标签
for i, region in enumerate(region_stats['Region']):
    plt.annotate(region, (region_pca[i, 0], region_pca[i, 1]), 
                fontsize=9, ha='center', va='bottom')

# 添加聚类中心
centers = pca.transform(final_kmeans.cluster_centers_)
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5, marker='X')

if chinese_font is not None:
    plt.title('地区聚类分布 (PCA降维)', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('主成分1', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('主成分2', fontproperties=chinese_font, fontsize=12)
    plt.legend(*scatter.legend_elements(), title='聚类', prop=chinese_font)
else:
    plt.title('地区聚类分布 (PCA降维)', fontsize=16)
    plt.xlabel('主成分1', fontsize=12)
    plt.ylabel('主成分2', fontsize=12)
    plt.legend(*scatter.legend_elements(), title='聚类')

plt.grid(True)
plt.tight_layout()
plt.savefig(f'{output_dir}/cluster_pca_visualization.png', dpi=300)
print(f"  保存图表：{output_dir}/cluster_pca_visualization.png")
plt.close()

# 3.4 分析每个聚类的风险分布
print("  分析聚类风险分布...")
cluster_risk = region_stats.groupby('Cluster')['Risk_Score'].agg(['mean', 'min', 'max', 'count'])
cluster_risk.columns = ['平均风险评分', '最小风险评分', '最大风险评分', '地区数量']
print(cluster_risk)

# 3.5 生成聚类特征描述
print("  生成聚类特征描述...")
# 创建一个聚类描述字典
cluster_descriptions = {}

for cluster in range(best_k):
    # 获取该聚类的平均特征
    cluster_data = cluster_means.loc[cluster]
    
    # 排序找出该聚类的最显著特征（最高值）
    top_features = cluster_data.sort_values(ascending=False).head(3)
    
    # 生成描述文本
    description = f"聚类 {cluster} 特点:\n"
    description += f"- 平均风险评分: {cluster_risk.loc[cluster, '平均风险评分']:.2f}\n"
    description += f"- 包含地区数量: {cluster_risk.loc[cluster, '地区数量']:.0f}\n"
    description += "- 显著特征:\n"
    
    for feature, value in top_features.items():
        description += f"  • {feature.replace('_', ' ').title()}: {value:.2f}\n"
    
    # 该聚类中的地区
    regions_in_cluster = region_stats[region_stats['Cluster'] == cluster]['Region'].tolist()
    description += f"- 包含地区: {', '.join(regions_in_cluster)}\n"
    
    cluster_descriptions[cluster] = description

# 保存聚类描述
with open(f'{output_dir}/cluster_descriptions.txt', 'w', encoding='utf-8') as f:
    for cluster, description in cluster_descriptions.items():
        f.write(f"{description}\n\n")

print(f"  聚类描述已保存至: {output_dir}/cluster_descriptions.txt")


3. 分析不同聚类的特点
  计算聚类特征...
  聚类平均特征值:
         Dengue_Cases_mean  Dengue_Cases_std  Dengue_Deaths_mean  \
Cluster                                                            
0              1932.766667       1936.956886           66.800000   
1               774.887500        857.140869            6.281250   
2              2176.245833       2458.518627           19.662500   
3               571.725000        503.732560           21.258333   

         Fatality_Rate  High_Season_Avg_Cases  Outbreak_Frequency  Risk_Score  
Cluster                                                                        
0             3.456185            3561.333333                0.05   54.434713  
1             0.800074            1411.500000                0.05   15.612254  
2             0.977676            4198.683333                0.05   47.788102  
3             3.396567             950.816667                0.05   24.786393  
  可视化聚类特征...
  保存图表：dengue_risk_analysis_results/cluster_radar_chart.png
 

In [46]:
# 1. 可视化区域风险分布
print("1. 可视化区域风险分布")

# 1.1 创建风险评分热图
print("  创建风险评分热图...")
# 重塑数据以便于热图可视化
# 提取每个地区每个月的平均病例数
monthly_avg_cases = df.groupby(['Region', 'MonthNum'])['Dengue_Cases'].mean().reset_index()
monthly_avg_cases_pivot = monthly_avg_cases.pivot(index='Region', columns='MonthNum', values='Dengue_Cases')

# 按风险评分排序
monthly_avg_cases_pivot = monthly_avg_cases_pivot.reindex(region_stats.sort_values('Risk_Score', ascending=False)['Region'])

# 生成热图
plt.figure(figsize=(14, 10))
ax = sns.heatmap(monthly_avg_cases_pivot, cmap='YlOrRd', annot=False, fmt='.0f')

# 添加风险类别标签
for i, region in enumerate(monthly_avg_cases_pivot.index):
    risk_score = region_stats[region_stats['Region'] == region]['Risk_Score'].values[0]
    risk_category = region_stats[region_stats['Region'] == region]['Risk_Category'].values[0]
    plt.text(-1.5, i + 0.5, f"{risk_score:.1f}", ha='right', va='center', fontproperties=chinese_font)
    plt.text(-0.5, i + 0.5, f"({risk_category})", ha='right', va='center', fontsize=8, fontproperties=chinese_font)

if chinese_font is not None:
    plt.title('按风险评分排序的地区月度登革热病例热图', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('月份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('地区 (风险评分)', fontproperties=chinese_font, fontsize=12)
    # 设置x轴标签为月份名称
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(np.arange(12) + 0.5, month_names, fontproperties=chinese_font, rotation=45)
else:
    plt.title('按风险评分排序的地区月度登革热病例热图', fontsize=16)
    plt.xlabel('月份', fontsize=12)
    plt.ylabel('地区 (风险评分)', fontsize=12)
    # 设置x轴标签为月份名称
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(np.arange(12) + 0.5, month_names, rotation=45, fontproperties=chinese_font)

plt.tight_layout()
plt.savefig(f'{output_dir}/risk_ordered_heatmap.png', dpi=300)
print(f"  保存图表：{output_dir}/risk_ordered_heatmap.png")
plt.close()

# 1.2 创建聚类风险分布图
print("  创建聚类风险分布图...")
plt.figure(figsize=(12, 8))

# 为每个聚类创建箱线图
sns.boxplot(x='Cluster', y='Risk_Score', data=region_stats, palette='Set3')

# 添加散点以显示每个地区
sns.stripplot(x='Cluster', y='Risk_Score', data=region_stats, 
             color='black', alpha=0.5, jitter=True)

# 为每个点添加地区标签
for i, row in region_stats.iterrows():
    plt.text(row['Cluster'], row['Risk_Score'], row['Region'], 
             fontsize=8, ha='center', va='bottom')

if chinese_font is not None:
    plt.title('各聚类的风险评分分布', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('聚类', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('风险评分', fontproperties=chinese_font, fontsize=12)
else:
    plt.title('各聚类的风险评分分布', fontsize=16)
    plt.xlabel('聚类', fontsize=12)
    plt.ylabel('风险评分', fontsize=12)

plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig(f'{output_dir}/cluster_risk_distribution.png', dpi=300)
print(f"  保存图表：{output_dir}/cluster_risk_distribution.png")
plt.close()

1. 可视化区域风险分布
  创建风险评分热图...
  保存图表：dengue_risk_analysis_results/risk_ordered_heatmap.png
  创建聚类风险分布图...
  保存图表：dengue_risk_analysis_results/cluster_risk_distribution.png


In [49]:
# 2. 分析高风险区域特征
print("\n2. 分析高风险区域特征")

# 2.1 提取高风险地区
print("  提取高风险地区...")
high_risk_regions = region_stats[region_stats['Risk_Category'].isin(['高风险', '极高风险'])]
print(f"  高风险地区数量: {len(high_risk_regions)}")
print("  高风险地区列表:")
for i, (index, row) in enumerate(high_risk_regions.iterrows()):
    print(f"    {i+1}. {row['Region']} - 风险评分: {row['Risk_Score']:.2f}, 类别: {row['Risk_Category']}")

# 2.2 分析高风险地区的季节性特点
print("  分析高风险地区的季节性特点...")
# 创建高风险地区的月度病例热图
high_risk_monthly = df[df['Region'].isin(high_risk_regions['Region'])]
high_risk_monthly = high_risk_monthly.groupby(['Region', 'MonthNum'])['Dengue_Cases'].mean().reset_index()
high_risk_monthly_pivot = high_risk_monthly.pivot(index='Region', columns='MonthNum', values='Dengue_Cases')

plt.figure(figsize=(14, 6))
ax = sns.heatmap(high_risk_monthly_pivot, cmap='YlOrRd', annot=True, fmt='.0f')

if chinese_font is not None:
    plt.title('高风险地区的月度登革热病例分布', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('月份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('高风险地区', fontproperties=chinese_font, fontsize=12)
    # 设置x轴标签为月份名称
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(np.arange(12) + 0.5, month_names, fontproperties=chinese_font, rotation=45)
else:
    plt.title('高风险地区的月度登革热病例分布', fontsize=16)
    plt.xlabel('月份', fontsize=12)
    plt.ylabel('高风险地区', fontsize=12)
    # 设置x轴标签为月份名称
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(np.arange(12) + 0.5, month_names, rotation=45)

plt.tight_layout()
plt.savefig(f'{output_dir}/high_risk_seasonal_pattern.png', dpi=300)
print(f"  保存图表：{output_dir}/high_risk_seasonal_pattern.png")
plt.close()

# 2.3 统计高风险地区的关键指标
print("  统计高风险地区的关键指标...")
high_risk_stats = high_risk_regions[cluster_features + ['Risk_Score', 'Risk_Category']]
high_risk_stats_mean = high_risk_stats.mean(numeric_only=True)
high_risk_stats_std = high_risk_stats.std(numeric_only=True)

# 比较高风险地区和全部地区的指标
all_regions_stats = region_stats[cluster_features + ['Risk_Score']]
all_regions_mean = all_regions_stats.mean()

comparison_df = pd.DataFrame({
    '高风险地区平均值': high_risk_stats_mean,
    '全部地区平均值': all_regions_mean,
    '差异倍数': high_risk_stats_mean / all_regions_mean
})

print("  高风险地区与全部地区指标比较:")
print(comparison_df)

# 保存比较结果
comparison_df.to_csv(f'{output_dir}/high_risk_vs_all_comparison.csv')
print(f"  比较结果已保存至: {output_dir}/high_risk_vs_all_comparison.csv")

# 2.4 可视化高风险地区与全部地区的对比
print("  可视化高风险地区与全部地区的对比...")
plt.figure(figsize=(14, 8))

# 对比图
bars = plt.barh(comparison_df.index, comparison_df['差异倍数'], color='darkred')

# 添加标签
for i, bar in enumerate(bars):
    plt.text(bar.get_width() + 0.1, i, f"{comparison_df['差异倍数'].iloc[i]:.2f}倍", va='center', fontproperties=chinese_font)

if chinese_font is not None:
    plt.title('高风险地区相比全部地区的指标差异（倍数）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('差异倍数', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('指标', fontproperties=chinese_font, fontsize=12)
else:
    plt.title('高风险地区相比全部地区的指标差异（倍数）', fontsize=16)
    plt.xlabel('差异倍数', fontsize=12)
    plt.ylabel('指标', fontsize=12)

plt.axvline(x=1, color='gray', linestyle='--')  # 添加参考线
plt.grid(True, axis='x')
plt.tight_layout()
plt.savefig(f'{output_dir}/high_risk_vs_all_comparison.png', dpi=300)
print(f"  保存图表：{output_dir}/high_risk_vs_all_comparison.png")
plt.close()


2. 分析高风险区域特征
  提取高风险地区...
  高风险地区数量: 1
  高风险地区列表:
    1. Region IV-A - 风险评分: 60.76, 类别: 高风险
  分析高风险地区的季节性特点...
  保存图表：dengue_risk_analysis_results/high_risk_seasonal_pattern.png
  统计高风险地区的关键指标...
  高风险地区与全部地区指标比较:
                          高风险地区平均值      全部地区平均值      差异倍数
Dengue_Cases_mean      2717.150000  1124.926471  2.415402
Dengue_Cases_std       3485.744342  1214.299329  2.870581
Dengue_Deaths_mean       10.866667    16.513725  0.658038
Fatality_Rate             0.399929     1.609044  0.248551
High_Season_Avg_Cases  5235.400000  2085.372549  2.510535
Outbreak_Frequency        0.050000     0.050000  1.000000
Risk_Score               60.760408    27.625337  2.199445
  比较结果已保存至: dengue_risk_analysis_results/high_risk_vs_all_comparison.csv
  可视化高风险地区与全部地区的对比...
  保存图表：dengue_risk_analysis_results/high_risk_vs_all_comparison.png


In [48]:
# 3. 准备区域风险报告
print("\n3. 准备区域风险报告")

# 3.1 创建综合风险报告
print("  创建综合风险报告...")
# 创建Markdown格式的报告
report = "# 菲律宾登革热区域风险分析报告\n\n"
report += "## 1. 总体风险评估\n\n"
report += f"* 分析时间范围: {df['Date'].min().strftime('%Y-%m-%d')} 至 {df['Date'].max().strftime('%Y-%m-%d')}\n"
report += f"* 分析地区数量: {len(region_stats)}\n"
report += f"* 高风险地区数量: {len(high_risk_regions)}\n\n"

report += "## 2. 风险评分分布\n\n"
risk_distribution = region_stats['Risk_Category'].value_counts().sort_index()
report += "风险类别分布:\n\n"
for category, count in risk_distribution.items():
    report += f"* {category}: {count}个地区 ({count/len(region_stats)*100:.1f}%)\n"

report += "\n## 3. 高风险地区列表\n\n"
report += "| 排名 | 地区 | 风险评分 | 风险类别 | 聚类 |\n"
report += "|------|------|----------|----------|------|\n"
for rank, (index, row) in enumerate(high_risk_regions.iterrows()):
    report += f"| {rank+1} | {row['Region']} | {row['Risk_Score']:.2f} | {row['Risk_Category']} | 聚类{row['Cluster']} |\n"

report += "\n## 4. 聚类分析\n\n"
report += f"* 最佳聚类数: {best_k}\n\n"
for cluster in range(best_k):
    report += f"### 聚类 {cluster}\n\n"
    report += cluster_descriptions[cluster].replace('\n', '\n\n') + "\n"

report += "\n## 5. 高风险地区特征\n\n"
report += "高风险地区与全部地区指标比较:\n\n"
report += "| 指标 | 高风险地区平均值 | 全部地区平均值 | 差异倍数 |\n"
report += "|------|-----------------|--------------|----------|\n"
for index, row in comparison_df.iterrows():
    report += f"| {index} | {row['高风险地区平均值']:.2f} | {row['全部地区平均值']:.2f} | {row['差异倍数']:.2f}倍 |\n"

report += "\n## 6. 高风险地区季节性模式\n\n"
report += "高风险地区普遍在以下月份达到高峰:\n\n"
# 找出高风险地区的高发月份
high_risk_monthly_mean = high_risk_monthly.groupby('MonthNum')['Dengue_Cases'].mean()
top3_months = high_risk_monthly_mean.sort_values(ascending=False).head(3)
for month, value in top3_months.items():
    month_name = month_names[month-1]
    report += f"* {month_name}: 平均 {value:.2f} 病例\n"

report += "\n## 7. 风险防控建议\n\n"
report += "1. **重点监测高风险地区**：集中资源对高风险地区进行更频繁的监测和预警。\n\n"
report += "2. **季节性防控**：在7-10月高发季节前加强防控措施。\n\n"
report += "3. **聚类化管理**：根据聚类特征设计针对性防控策略：\n\n"

for cluster in range(best_k):
    cluster_risk_level = cluster_risk.loc[cluster, '平均风险评分']
    if cluster_risk_level > 60:  # 高风险聚类
        report += f"   * 聚类 {cluster}（高风险）：加强社区宣传和蚊虫防控，增加医疗资源投入。\n\n"
    elif cluster_risk_level > 40:  # 中等风险聚类
        report += f"   * 聚类 {cluster}（中等风险）：定期监测，季节性增强防控。\n\n"
    else:  # 低风险聚类
        report += f"   * 聚类 {cluster}（低风险）：维持基本监测和防控措施。\n\n"

report += "4. **跨区域协作**：相同聚类的地区可共享防控资源和经验。\n\n"

# 保存报告
with open(f'{output_dir}/risk_analysis_report.md', 'w', encoding='utf-8') as f:
    f.write(report)
print(f"  风险分析报告已保存至: {output_dir}/risk_analysis_report.md")

# 3.2 创建地区风险卡片
print("  创建地区风险卡片...")
# 为每个地区创建风险卡片
for i, (index, row) in enumerate(region_stats.iterrows()):
    region = row['Region']
    
    # 创建地区风险卡片报告
    card = f"# {region} 登革热风险卡片\n\n"
    card += f"## 风险评估\n\n"
    card += f"* **风险评分**: {row['Risk_Score']:.2f}/100\n"
    card += f"* **风险类别**: {row['Risk_Category']}\n"
    card += f"* **聚类分类**: 聚类 {row['Cluster']}\n\n"
    
    card += f"## 关键指标\n\n"
    card += f"* **平均病例数**: {row['Dengue_Cases_mean']:.2f}\n"
    card += f"* **最大病例数**: {row['Dengue_Cases_max']:.0f}\n"
    card += f"* **病例波动性**: {row['Dengue_Cases_std']:.2f}\n"
    card += f"* **病死率**: {row['Fatality_Rate']:.2f}%\n"
    card += f"* **高发季平均病例**: {row['High_Season_Avg_Cases']:.2f}\n"
    card += f"* **爆发频率**: {row['Outbreak_Frequency']*100:.2f}%\n\n"
    
    # 提取该地区的季节性模式
    region_monthly = df[df['Region'] == region].groupby('MonthNum')['Dengue_Cases'].mean()
    top_months = region_monthly.sort_values(ascending=False).head(3)
    
    card += f"## 季节性模式\n\n"
    card += f"高发月份:\n\n"
    for month, value in top_months.items():
        month_name = month_names[month-1]
        card += f"* {month_name}: 平均 {value:.2f} 病例\n"
    
    card += f"\n## 防控建议\n\n"
    
    # 根据风险类别生成建议
    if row['Risk_Category'] in ['高风险', '极高风险']:
        card += "1. **加强监测**: 建立常规监测系统，密切追踪病例。\n"
        card += "2. **资源优先**: 优先分配防控资源和医疗设备。\n"
        card += "3. **社区参与**: 加强社区健康教育和预防宣传。\n"
        card += "4. **早期干预**: 在高发季前2个月加强防控措施。\n"
    elif row['Risk_Category'] == '中等风险':
        card += "1. **季节性监测**: 在高发季节加强监测。\n"
        card += "2. **选择性干预**: 针对高风险点实施防控措施。\n"
        card += "3. **提高意识**: 开展社区健康教育活动。\n"
    else:
        card += "1. **基础监测**: 维持基本的监测系统。\n"
        card += "2. **风险评估**: 定期更新风险状况。\n"
        card += "3. **公众教育**: 基本的健康知识宣传。\n"
    
    # 保存地区风险卡片
    with open(f'{output_dir}/{region}_risk_card.md', 'w', encoding='utf-8') as f:
        f.write(card)

print(f"  已为{len(region_stats)}个地区创建风险卡片")


3. 准备区域风险报告
  创建综合风险报告...
  风险分析报告已保存至: dengue_risk_analysis_results/risk_analysis_report.md
  创建地区风险卡片...
  已为17个地区创建风险卡片
